In [ ]:
from confluent_kafka import Consumer, KafkaError
import pandas as pd
from threading import Timer
import time
from datetime import datetime

settings = {
    'bootstrap.servers': 'localhost:9092',
    'group.id': 'mygroup',
    'client.id': 'client-1',
    'enable.auto.commit': True,
    'session.timeout.ms': 6000,
    'default.topic.config': {'auto.offset.reset': 'smallest'}
}

def s(a):
    aa=[]
    for i in a:
        for j in range(len(i)):
            if i[j]=="#":
                c=0
                for jj in range(j,len(i)):
                    if i[jj]=="l":
                        c=c+1
                    if c==1:
                        t=i[jj:-24]
                        t=t.split("\n")
                        aa.append(t)
                        
                        break
    aaa=[]
    for ii in aa:
        tem=[]
        for i in ii:
            
            j=0
            f=0
            while j<len(i):
                
                if i[j]==" ":
                    while i[j]==" ":
                    
                        j=j+1
                        f=1

                if f==1:
                    break
                j=j+1
            tt=i[j:]
            
            if tt:
                tt=float(tt)
               
                tem.append(tt)

        aaa.append(tem)
    df = pd.DataFrame(aaa,columns = ['label', 'ACC_x', 'ACC_y', 'ACC_z', 'ECG', 'EMG', 'Temp', 'Resp']) 
    return df


from pyspark import SparkContext
from pyspark.sql import SparkSession
sc = SparkContext('local', 'myapp')
spark = SparkSession.builder.getOrCreate()
from pyspark.mllib.tree import RandomForestModel
sameModel = RandomForestModel.load(sc, "myRandomForestClassificationModel")




while True:
    print(datetime.now())
    starttime=datetime.now()
    c = Consumer(settings)
    c.subscribe(['mytopic'])
    a=[]
    qq=0

    while qq<3500:
        try:


            msg = c.poll(0.1)
            if msg is None:
                continue
            elif not msg.error():
                a.append(msg.value().decode('utf-8'))
            elif msg.error().code() == KafkaError._PARTITION_EOF:
                print('End of partition reached {0}/{1}'
                        .format(msg.topic(), msg.partition()))
            else:
                print('Error occured: {0}'.format(msg.error().str()))

            qq=qq+1

        except KeyboardInterrupt:
            pass
            qq=qq+1

    df=s(a)

    sdf = spark.createDataFrame(df)
    from pyspark.mllib.linalg import Vectors
    from pyspark.mllib.regression import LabeledPoint
    test_data = sdf.rdd.map(lambda row: LabeledPoint(row[0], Vectors.dense(row[1:])))
    predictions = sameModel.predict(test_data.map(lambda x: x.features))
    pq=predictions.collect()
   
    d={}
    for i in pq:
        if i in d:
            d[i]+=1
        else:
            d[i]=1

  
    temp=max(d,key=d.get)
    print(temp)
    endtime=datetime.now()
    tt=endtime-starttime
    tt=str(tt)
    tt=tt[5:]
    tt=float(tt)

    
    st=5-tt

    if st>0:
        time.sleep(st)
    else:
        print(st)

    print('###############')
    df = pd.read_csv('2245nnw.csv')
    b1=df.loc[1:700, ['Temp']].mean()
    t1=b1[0]
    t1=float(str(t1))
    b2=df.loc[701:1400, ['Temp']].mean()
    t2=b2[0]
    t2=float(str(t2))
    b3=df.loc[1401:2100, ['Temp']].mean()
    t3=b3[0]
    t3=float(str(t3))
    b4=df.loc[2101:2800, ['Temp']].mean()
    t4=b4[0]
    t4=float(str(t4))
    b5=df.loc[2801:3500, ['Temp']].mean()
    t5=b5[0]
    t5=float(str(t5))
    bb1=df.loc[1:700, ['ECG']].mean()
    tt1=bb1[0]
    tt1=float(str(tt1))
    bb2=df.loc[701:1400, ['ECG']].mean()
    tt2=bb2[0]
    tt2=float(str(tt2))
    bb3=df.loc[1401:2100, ['ECG']].mean()
    tt3=bb3[0]
    tt3=float(str(tt3))
    bb4=df.loc[2101:2800, ['ECG']].mean()
    tt4=bb4[0]
    tt4=float(str(tt4))
    bb5=df.loc[2801:3500, ['ECG']].mean()
    tt5=bb5[0]
    tt5=float(str(tt5))
    g1=[temp,t1,tt1]
    g2=[temp,t2,tt2]
    g3=[temp,t3,tt3]
    g4=[temp,t4,tt4]
    g5=[temp,t5,tt5]
    k=[]
    k.append(g1)
    k.append(g2)
    k.append(g3)
    k.append(g4)
    k.append(g5)
    gg=pd.DataFrame(k)
    gg.to_csv("falsk.csv",index=False,sep=',')